# Cloud Carbon Coefficients

#### Imports

In [1]:
%pip install -r requirements.txt

import csv
import numpy as np
import pandas as pd

  Using cached pandas-1.2.5-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (9.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
ERROR: Could not install packages due to an OSError: [Errno 30] Read-only file system: 'INSTALLER'

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Processor types

Processors are grouped into types by each vendor e.g. Intel Broadwell CPUs. Cloud providers provide the CPU types for each of their instance types, but not the precise CPU details. As such, we calculate the average wattage by processor type.

In [2]:
# Loads a CSV file then returns each row appended to an array
def load_append_data(file_name):
    with open(f'data/{file_name}', 'r') as csvfile:
        reader = csv.reader(csvfile)

        data = []
        for row in reader:
            data.append(row[0])
        
        return data

cpus_amd_epyc_gen1 = load_append_data('amd-epyc-gen1.csv')
cpus_amd_epyc_gen2 = load_append_data('amd-epyc-gen2.csv')
cpus_intel_sandybridge = load_append_data('intel-sandybridge.csv')
cpus_intel_ivybridge = load_append_data('intel-ivybridge.csv')
cpus_intel_haswell = load_append_data('intel-haswell.csv')
cpus_intel_broadwell = load_append_data('intel-broadwell.csv')
cpus_intel_skylake = load_append_data('intel-skylake.csv')
cpus_intel_cascadelake = load_append_data('intel-cascadelake.csv')
cpus_intel_coffeelake = load_append_data('intel-coffeelake.csv')

## Processor lists

Now we know which processors are in which type, we can group all the tested servers by their CPU type to calculate: average idle watts, average watts at 100% utilization, average GB/chip.

In [3]:
# Load all servers from SPECpower results CSV
servers = pd.read_csv('data/SPECpower-full-results.csv', na_values=['NC'])

#### Regex match

The regex to match the CPU names matches to the end of the line using `$` because some chips have version numbers after, so we can't just do a substring match e.g. `Intel E3-1230` is a Sandy Bridge chip but `Intel E3-1230 v3` is Haswell. It is case insensitive due to the use of `v3` and `V3` inconsistently in the definitions.

#### Clean data

The SPECpower results often appends extra info to the `CPU Description` column which is unecessary. For example, `Intel Xeon E5-2470 (Intel Turbo Boost Technology up to 3.10 GHz)`. This extra info needs to be stripped e.g. to `Intel Xeon E5-2470` otherwise the regex match will not work.

The check below will error if the data is not clean.

In [4]:
if len(servers[servers['CPU Description'].str.contains('Ghz')]) > 0:
    print('Data not clean')
    import sys
    sys.exit(1)

### AMD: EPYC Gen 1

In [5]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_amd_epyc_gen1]
servers_amd_epyc_gen1 = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

amd_epyc_gen1 = {}
amd_epyc_gen1['Idle watts'] = (servers_amd_epyc_gen1['avg. watts @ active idle'].astype(float) / servers_amd_epyc_gen1['Total Threads']).mean()
amd_epyc_gen1['100% watts'] = (servers_amd_epyc_gen1['avg. watts @ 100%'].astype(float) / servers_amd_epyc_gen1['Total Threads']).mean()
amd_epyc_gen1['GB/Chip'] = (servers_amd_epyc_gen1['Total Memory (GB)'] / servers_amd_epyc_gen1['Chips']).mean()

print(f'Average: Min Watts = {amd_epyc_gen1["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {amd_epyc_gen1["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {amd_epyc_gen1["GB/Chip"].mean():,.2f}')

Average: Min Watts = 0.82
Average: Max Watts = 2.55
Average: GB/Chip = 89.60
/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


### AMD: EPYC Gen 2

In [6]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_amd_epyc_gen2]
servers_amd_epyc_gen2 = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

amd_epyc_gen2 = {}
amd_epyc_gen2['Idle watts'] = (servers_amd_epyc_gen2['avg. watts @ active idle'].astype(float) / servers_amd_epyc_gen2['Total Threads']).mean()
amd_epyc_gen2['100% watts'] = (servers_amd_epyc_gen2['avg. watts @ 100%'].astype(float) / servers_amd_epyc_gen2['Total Threads']).mean()
amd_epyc_gen2['GB/Chip'] = (servers_amd_epyc_gen2['Total Memory (GB)'] / servers_amd_epyc_gen2['Chips']).mean()

print(f'Average: Min Watts = {amd_epyc_gen2["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {amd_epyc_gen2["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {amd_epyc_gen2["GB/Chip"].mean():,.2f}')

Average: Min Watts = 0.47
Average: Max Watts = 1.69
Average: GB/Chip = 129.78


### Intel: Sandy Bridge

In [7]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_sandybridge]
servers_intel_sandybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_sandybridge = {}
intel_sandybridge['Idle watts'] = (servers_intel_sandybridge['avg. watts @ active idle'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['100% watts'] = (servers_intel_sandybridge['avg. watts @ 100%'].astype(float) / servers_intel_sandybridge['Total Threads']).mean()
intel_sandybridge['GB/Chip'] = (servers_intel_sandybridge['Total Memory (GB)'] / servers_intel_sandybridge['Chips']).mean()

print(f'Average: Min Watts = {intel_sandybridge["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_sandybridge["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_sandybridge["GB/Chip"].mean():,.2f}')

Average: Min Watts = 2.17
Average: Max Watts = 8.58
Average: GB/Chip = 16.48


### Intel: Ivy Bridge

In [8]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_ivybridge]
servers_intel_ivybridge = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_ivybridge = {}
intel_ivybridge['Idle watts'] = (servers_intel_ivybridge['avg. watts @ active idle'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['100% watts'] = (servers_intel_ivybridge['avg. watts @ 100%'].astype(float) / servers_intel_ivybridge['Total Threads']).mean()
intel_ivybridge['GB/Chip'] = (servers_intel_ivybridge['Total Memory (GB)'] / servers_intel_ivybridge['Chips']).mean()

print(f'Average: Min Watts = {intel_ivybridge["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_ivybridge["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_ivybridge["GB/Chip"].mean():,.2f}')

Average: Min Watts = 3.04
Average: Max Watts = 8.25
Average: GB/Chip = 14.93


### Intel: Haswell

In [9]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_haswell]
servers_intel_haswell = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_haswell = {}
intel_haswell['Idle watts'] = (servers_intel_haswell['avg. watts @ active idle'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['100% watts'] = (servers_intel_haswell['avg. watts @ 100%'].astype(float) / servers_intel_haswell['Total Threads']).mean()
intel_haswell['GB/Chip'] = (servers_intel_haswell['Total Memory (GB)'] / servers_intel_haswell['Chips']).mean()

print(f'Average: Min Watts = {intel_haswell["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_haswell["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_haswell["GB/Chip"].mean():,.2f}')

Average: Min Watts = 1.00
Average: Max Watts = 4.74
Average: GB/Chip = 27.71


### Intel: Broadwell

In [10]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_broadwell]
servers_intel_broadwell = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_broadwell = {}
intel_broadwell['Idle watts'] = (servers_intel_broadwell['avg. watts @ active idle'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['100% watts'] = (servers_intel_broadwell['avg. watts @ 100%'].astype(float) / servers_intel_broadwell['Total Threads']).mean()
intel_broadwell['GB/Chip'] = (servers_intel_broadwell['Total Memory (GB)'] / servers_intel_broadwell['Chips']).mean()

print(f'Average: Min Watts = {intel_broadwell["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_broadwell["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_broadwell["GB/Chip"].mean():,.2f}')

Average: Min Watts = 0.71
Average: Max Watts = 3.69
Average: GB/Chip = 69.65


### Intel: Skylake

In [11]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_skylake]
servers_intel_skylake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_skylake = {}
intel_skylake['Idle watts'] = (servers_intel_skylake['avg. watts @ active idle'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['100% watts'] = (servers_intel_skylake['avg. watts @ 100%'].astype(float) / servers_intel_skylake['Total Threads']).mean()
intel_skylake['GB/Chip'] = (servers_intel_skylake['Total Memory (GB)'] / servers_intel_skylake['Chips']).mean()

print(f'Average: Min Watts = {intel_skylake["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_skylake["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_skylake["GB/Chip"].mean():,.2f}')

Average: Min Watts = 0.65
Average: Max Watts = 4.26
Average: GB/Chip = 81.32


### Intel: Cascade Lake

In [12]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_cascadelake]
servers_intel_cascadelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_cascadelake = {}
intel_cascadelake['Idle watts'] = (servers_intel_cascadelake['avg. watts @ active idle'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['100% watts'] = (servers_intel_cascadelake['avg. watts @ 100%'].astype(float) / servers_intel_cascadelake['Total Threads']).mean()
intel_cascadelake['GB/Chip'] = (servers_intel_cascadelake['Total Memory (GB)'] / servers_intel_cascadelake['Chips']).mean()

print(f'Average: Min Watts = {intel_cascadelake["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_cascadelake["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_cascadelake["GB/Chip"].mean():,.2f}')

Average: Min Watts = 0.64
Average: Max Watts = 3.97
Average: GB/Chip = 98.12


### Intel: Coffee Lake

In [13]:
# Construct regex to match the chip name exactly to the end of the line
# (See notes above on regex and clean data)
cpus_re = [rf'(?i)(\b{string}$)' for string in cpus_intel_coffeelake]
servers_intel_coffeelake = servers[servers['CPU Description'].str.contains('|'.join(cpus_re))]

intel_coffeelake = {}
intel_coffeelake['Idle watts'] = (servers_intel_coffeelake['avg. watts @ active idle'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['100% watts'] = (servers_intel_coffeelake['avg. watts @ 100%'].astype(float) / servers_intel_coffeelake['Total Threads']).mean()
intel_coffeelake['GB/Chip'] = (servers_intel_coffeelake['Total Memory (GB)'] / servers_intel_coffeelake['Chips']).mean()

print(f'Average: Min Watts = {intel_coffeelake["Idle watts"].mean():,.2f}')
print(f'Average: Max Watts = {intel_coffeelake["100% watts"].mean():,.2f}')
print(f'Average: GB/Chip = {intel_coffeelake["GB/Chip"].mean():,.2f}')

Average: Min Watts = 1.14
Average: Max Watts = 5.42
Average: GB/Chip = 19.56


## Azure

These values go in [`packages/azure/src/domain/AzureFootprintEstimationConstants.ts`](https://github.com/cloud-carbon-footprint/cloud-carbon-footprint/blob/trunk/packages/azure/src/domain/AzureFootprintEstimationConstants.ts)

In [14]:
azure_instances = pd.read_csv('data/azure-instances.csv', na_values=['NC'])
azure_architectures = azure_instances['Microarchitecture'].unique()
azure_coefficients = []

for cpu in azure_architectures:
    if cpu == 'AMD EPYC 1st Gen':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen1['Idle watts'],
            'Max Watts': amd_epyc_gen1['100% watts'],
            'GB/Chip': amd_epyc_gen1['GB/Chip']
        })
    elif cpu == 'AMD EPYC 2nd Gen':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen2['Idle watts'],
            'Max Watts': amd_epyc_gen2['100% watts'],
            'GB/Chip': amd_epyc_gen2['GB/Chip']
        })
    elif cpu == 'Sandy Bridge':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_sandybridge['Idle watts'],
            'Max Watts': intel_sandybridge['100% watts'],
            'GB/Chip': intel_sandybridge['GB/Chip']
        })
    elif cpu == 'Ivy Bridge':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_ivybridge['Idle watts'],
            'Max Watts': intel_ivybridge['100% watts'],
            'GB/Chip': intel_ivybridge['GB/Chip']
        })
    elif cpu == 'Haswell':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_haswell['Idle watts'],
            'Max Watts': intel_haswell['100% watts'],
            'GB/Chip': intel_haswell['GB/Chip']
        })
    elif cpu == 'Broadwell':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_broadwell['Idle watts'],
            'Max Watts': intel_broadwell['100% watts'],
            'GB/Chip': intel_broadwell['GB/Chip']
        })
    elif cpu == 'Skylake':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_skylake['Idle watts'],
            'Max Watts': intel_skylake['100% watts'],
            'GB/Chip': intel_skylake['GB/Chip']
        })
    elif cpu == 'Cascade Lake':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_cascadelake['Idle watts'],
            'Max Watts': intel_cascadelake['100% watts'],
            'GB/Chip': intel_cascadelake['GB/Chip']
        })
    elif cpu == 'Coffee Lake':
        azure_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_coffeelake['Idle watts'],
            'Max Watts': intel_coffeelake['100% watts'],
            'GB/Chip': intel_coffeelake['GB/Chip']
        })    

azure_coefficients = pd.DataFrame(azure_coefficients)
print(f'Average: Min Watts = {azure_coefficients["Min Watts"].mean():,.2f}')
print(f'Average: Max Watts = {azure_coefficients["Max Watts"].mean():,.2f}')
print(f'Average: GB/Chip = {azure_coefficients["GB/Chip"].mean():,.2f}')
azure_coefficients

Average: Min Watts = 0.78
Average: Max Watts = 3.76
Average: GB/Chip = 73.68


,Architecture,Min Watts,Max Watts,GB/Chip
0,Cascade Lake,0.638949,3.967305,98.117647
1,Skylake,0.652273,4.255506,81.324324
2,Broadwell,0.712834,3.685328,69.647059
3,Haswell,1.000595,4.739716,27.714286
4,AMD EPYC 2nd Gen,0.474262,1.692962,129.777778
5,Coffee Lake,1.138426,5.421759,19.555556
6,AMD EPYC 1st Gen,0.822656,2.553125,89.600000


## AWS

These values go in [`packages/aws/src/domain/AwsFootprintEstimationConstants.ts`](https://github.com/cloud-carbon-footprint/cloud-carbon-footprint/blob/trunk/packages/aws/src/domain/AwsFootprintEstimationConstants.ts)

In [15]:
aws_instances = pd.read_csv('data/aws-instances.csv', na_values=['NC'])
aws_architectures = aws_instances['Microarchitecture'].unique()
aws_coefficients = []

for cpu in aws_architectures:
    if cpu == 'AMD EPYC 1st Gen':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen1['Idle watts'],
            'Max Watts': amd_epyc_gen1['100% watts'],
            'GB/Chip': amd_epyc_gen1['GB/Chip']
        })
    elif cpu == 'AMD EPYC 2nd Gen':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen2['Idle watts'],
            'Max Watts': amd_epyc_gen2['100% watts'],
            'GB/Chip': amd_epyc_gen2['GB/Chip']
        })
    # We don't know the values for the Graviton chips so
    # assume they are the same as AMD EPYC Gen 2
    elif cpu == 'AWS Graviton2':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen2['Idle watts'],
            'Max Watts': amd_epyc_gen2['100% watts'],
            'GB/Chip': amd_epyc_gen2['GB/Chip']
        })
    elif cpu == 'Sandy Bridge':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_sandybridge['Idle watts'],
            'Max Watts': intel_sandybridge['100% watts'],
            'GB/Chip': intel_sandybridge['GB/Chip']
        })
    elif cpu == 'Ivy Bridge':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_ivybridge['Idle watts'],
            'Max Watts': intel_ivybridge['100% watts'],
            'GB/Chip': intel_ivybridge['GB/Chip']
        })
    elif cpu == 'Haswell':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_haswell['Idle watts'],
            'Max Watts': intel_haswell['100% watts'],
            'GB/Chip': intel_haswell['GB/Chip']
        })
    elif cpu == 'Broadwell':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_broadwell['Idle watts'],
            'Max Watts': intel_broadwell['100% watts'],
            'GB/Chip': intel_broadwell['GB/Chip']
        })
    elif cpu == 'Skylake':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_skylake['Idle watts'],
            'Max Watts': intel_skylake['100% watts'],
            'GB/Chip': intel_skylake['GB/Chip']
        })
    elif cpu == 'Cascade Lake':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_cascadelake['Idle watts'],
            'Max Watts': intel_cascadelake['100% watts'],
            'GB/Chip': intel_cascadelake['GB/Chip']
        })
    elif cpu == 'Coffee Lake':
        aws_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_coffeelake['Idle watts'],
            'Max Watts': intel_coffeelake['100% watts'],
            'GB/Chip': intel_coffeelake['GB/Chip']
        })

aws_coefficients = pd.DataFrame(aws_coefficients)
print(f'Average: Min Watts = {aws_coefficients["Min Watts"].mean():,.2f}')
print(f'Average: Max Watts = {aws_coefficients["Max Watts"].mean():,.2f}')
print(f'Average: GB/Chip = {aws_coefficients["GB/Chip"].mean():,.2f}')
aws_coefficients

Average: Min Watts = 0.74
Average: Max Watts = 3.50
Average: GB/Chip = 80.69


,Architecture,Min Watts,Max Watts,GB/Chip
0,Haswell,1.000595,4.739716,27.714286
1,Cascade Lake,0.638949,3.967305,98.117647
2,Skylake,0.652273,4.255506,81.324324
3,AMD EPYC 2nd Gen,0.474262,1.692962,129.777778
4,AWS Graviton2,0.474262,1.692962,129.777778
5,Broadwell,0.712834,3.685328,69.647059
6,AMD EPYC 1st Gen,0.822656,2.553125,89.600000
7,Coffee Lake,1.138426,5.421759,19.555556


## GCP

These values go in [`packages/gcp/src/domain/GcpFootprintEstimationConstants.ts`](https://github.com/cloud-carbon-footprint/cloud-carbon-footprint/blob/trunk/packages/gcp/src/domain/GcpFootprintEstimationConstants.ts)

In [67]:
gcp_instances = pd.read_csv('data/gcp-instances.csv', na_values=['NC'])
gcp_architectures = gcp_instances['Microarchitecture'].unique()
gcp_coefficients = []

for cpu in gcp_architectures:
    if cpu == 'AMD EPYC 1st Gen':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen1['Idle watts'],
            'Max Watts': amd_epyc_gen1['100% watts'],
            'GB/Chip': amd_epyc_gen1['GB/Chip']
        })
    elif cpu == 'AMD EPYC 2nd Gen':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': amd_epyc_gen2['Idle watts'],
            'Max Watts': amd_epyc_gen2['100% watts'],
            'GB/Chip': amd_epyc_gen2['GB/Chip']
        })
    elif cpu == 'Sandy Bridge':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_sandybridge['Idle watts'],
            'Max Watts': intel_sandybridge['100% watts'],
            'GB/Chip': intel_sandybridge['GB/Chip']
        })
    elif cpu == 'Ivy Bridge':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_ivybridge['Idle watts'],
            'Max Watts': intel_ivybridge['100% watts'],
            'GB/Chip': intel_ivybridge['GB/Chip']
        })
    elif cpu == 'Haswell':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_haswell['Idle watts'],
            'Max Watts': intel_haswell['100% watts'],
            'GB/Chip': intel_haswell['GB/Chip']
        })
    elif cpu == 'Broadwell':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_broadwell['Idle watts'],
            'Max Watts': intel_broadwell['100% watts'],
            'GB/Chip': intel_broadwell['GB/Chip']
        })
    elif cpu == 'Skylake':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_skylake['Idle watts'],
            'Max Watts': intel_skylake['100% watts'],
            'GB/Chip': intel_skylake['GB/Chip']
        })
    elif cpu == 'Cascade Lake':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_cascadelake['Idle watts'],
            'Max Watts': intel_cascadelake['100% watts'],
            'GB/Chip': intel_cascadelake['GB/Chip']
        })
    elif cpu == 'Coffee Lake':
        gcp_coefficients.append({
            'Architecture': cpu,
            'Min Watts': intel_coffeelake['Idle watts'],
            'Max Watts': intel_coffeelake['100% watts'],
            'GB/Chip': intel_coffeelake['GB/Chip']
        })

gcp_coefficients = pd.DataFrame(gcp_coefficients)
print(f'Median: Min Watts = {gcp_coefficients["Min Watts"].median():,.2f}')
print(f'Median: Max Watts = {gcp_coefficients["Max Watts"].median():,.2f}')
gcp_coefficients

Median: Min Watts = 0.71
Median: Max Watts = 4.26


,Architecture,Min Watts,Max Watts,GB/Chip
0,Skylake,0.652273,4.255506,81.324324
1,Cascade Lake,0.638949,3.967305,98.117647
2,AMD EPYC 2nd Gen,0.474262,1.692962,129.777778
3,Broadwell,0.712834,3.685328,69.647059
4,Haswell,1.000595,4.739716,27.714286
5,Sandy Bridge,2.169441,8.575358,16.480916
6,Ivy Bridge,3.036927,8.248611,14.933333


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2bb85156-92ac-4c34-9a24-94671c8f593c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>